In [ ]:
!git clone https://github.com/huggingface/diffusers

In [ ]:
%cd diffusers

In [ ]:
!pip install -e .

In [ ]:
!pip install -r examples/dreambooth/requirements.txt

In [ ]:
!pip install bitsandbytes

In [ ]:
from accelerate.utils import write_basic_config
write_basic_config()

In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login(key = "wandb_api_key") # replace with your wandb key

In [ ]:
!accelerate launch examples/dreambooth/train_dreambooth.py \
    --pretrained_model_name_or_path="SG161222/Realistic_Vision_V6.0_B1_noVAE" \
    --instance_data_dir="image_data_dir" \
    --class_data_dir="image_data_dir" \
    --output_dir="/kaggle/working/dreambooth_output" \
    --with_prior_preservation \
    --prior_loss_weight=0.9 \
    --instance_prompt="a photo of owhn man" \
    --class_prompt="a photo of a man" \
    --resolution=512 \
    --train_batch_size=1 \
    --use_8bit_adam \
    --lr_scheduler="constant" \
    --lr_warmup_steps=0 \
    --gradient_accumulation_steps=1 \
    --learning_rate=5e-6 \
    --num_class_images=15 \
    --max_train_steps=1000 \
    --checkpointing_steps=500 \
    --report_to="wandb" \
    --logging_dir="logs" \
    --seed="0"

In [ ]:
%cd /kaggle/working
!rm -rf diffusers

In [11]:
import sys

# Xoá module diffusers khỏi cache
if "diffusers" in sys.modules:
    del sys.modules["diffusers"]

# Loại bỏ tất cả đường dẫn trỏ đến local diffusers
sys.path = [p for p in sys.path if "diffusers" not in p]

# Cài và import lại đúng diffusers

# Import lại

In [ ]:
!pip install git+https://github.com/huggingface/diffusers.git --quiet

In [ ]:
from diffusers import StableDiffusionPipeline
import torch
import os
from safetensors.torch import load_file
import numpy as np 
model_id = "/kaggle/working/dreambooth_output"
path_id = "SG161222/Realistic_Vision_V6.0_B1_noVAE"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")
# state_dict = load_file("/kaggle/working/dreambooth_output/checkpoint-1500/unet/diffusion_pytorch_model.safetensors")

# pipe.unet.load_state_dict(state_dict)
prompt = "(wide shot) of analog style modelshoot style 8k close up linkedin profile picture of owhn man, professional jack suite, professional headshots, photo-realistic, high-resolution image, workplace settings, upper body, modern outfit, blurred background, glass building. crisp details, neutral expression, high-definition, sharp focus"
negative_prompt = "(deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime), text, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck"
output_dir = "/kaggle/working/generated_images"
os.makedirs(output_dir, exist_ok=True)

for i in range(20):
    seed = np.random.randint(0, 999999)
    generator = torch.manual_seed(seed)

    image = pipe(prompt, negative_prompt = negative_prompt, num_inference_steps=50, guidance_scale=8, generator=generator).images[0]
    image.save(f"{output_dir}/image_{i+1:02d}_seed{seed}.png")